<a href="https://colab.research.google.com/github/MariiaKalinina/R/blob/main/func_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import torch
from math import *
from scipy.stats import pearsonr
from scipy import integrate
from numpy import linalg as LA
import matplotlib.pyplot as plt
%matplotlib inline
from numba import njit
import numpy as np
from scipy.integrate import dblquad

import matplotlib.pylab as plt
import seaborn as sns
import pandas as pd
from pathlib import Path
import matplotlib.colors

import matplotlib.pyplot as plt
import matplotlib.font_manager as font_manager
%matplotlib inline
csfont = {'fontname':'Times New Roman'}

In [ ]:

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

# Your imports
from mlxtend.plotting import plot_decision_regions

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier

from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.preprocessing import PolynomialFeatures, StandardScaler, QuantileTransformer, FunctionTransformer, MinMaxScaler
from sklearn.random_projection import GaussianRandomProjection
from sklearn.decomposition import PCA

from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error as mse

from sklearn.model_selection import GridSearchCV, PredefinedSplit

import time
import math
from sklearn.metrics import roc_curve, average_precision_score

import warnings
warnings.filterwarnings('ignore')

In [ ]:
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
import xgboost as xgb
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

In [ ]:
def model(X_train, y_train, X_val, y_val, X_test, y_test):

  # Assuming X_train, y_train, X_val, y_val, X_test, y_test are defined

  RMSE_validation, RMSE_test, RMSE_train = [], [], []
  R2_validation, R2_test, R2_train = [], [], []
  MAE_validation, MAE_test, MAE_train = [], [], []
  model_name = []

  # List of models to evaluate
  models_list = [
      RandomForestRegressor(random_state=0xC0FFEE),
      ExtraTreesRegressor(random_state=0xC0FFEE),
      xgb.XGBRegressor(random_state=0xC0FFEE),
      LinearRegression(),
      Ridge(),
      KNeighborsRegressor(),
      DecisionTreeRegressor(),
      GradientBoostingRegressor(random_state=0xC0FFEE),  # Added Gradient Boosting
      CatBoostRegressor(silent=True, random_state=0xC0FFEE),  # Added CatBoost
  ]

  # Parameter grids for each model
  p_g1 = {
      'n_estimators': np.arange(2, 200, 20),
      'min_samples_split': np.arange(2, 20, 2)
  }

  p_g2 = p_g1  # Same as p_g1 for simplicity

  p_g3 = {
      'n_estimators': np.arange(2, 200, 20),
      "learning_rate": np.linspace(0.005, 1, 10)
  }

  p_g4 = {  # Linear Regression has no hyperparameters to tune
  }

  p_g5 = {  # Ridge Regression parameters
      'alpha': np.logspace(-3, 3, 7),  # Regularization strength
  }

  p_g6 = {  # KNeighbors Regressor parameters
      'n_neighbors': np.arange(1, 21),   # Number of neighbors
      'weights': ['uniform', 'distance'], # Weight function used in prediction
  }

  p_g7 = {  # Decision Tree Regressor parameters
      'max_depth': [None] + list(np.arange(1, 11)),   # Maximum depth of the tree
      'min_samples_split': np.arange(2, 21),           # Minimum number of samples required to split an internal node
  }

  p_g8 = {  # Gradient Boosting parameters
      'n_estimators': np.arange(50, 301, 50),          # Number of boosting stages to be run
      'learning_rate': [0.01, 0.1, 0.2, 0.5],                # Step size shrinkage used in update to prevents overfitting
      'max_depth': np.arange(2, 12, 2),                           # Maximum depth of the individual regression estimators
  }

  p_g9 = {  # CatBoost parameters (basic)
      'iterations': [50, 100, 150, 200],              # Number of boosting iterations
      'learning_rate': np.logspace(-3, 0, 5),                     # Learning rate for boosting steps
      'depth': np.arange(2, 12, 2),                                  # Depth of the tree
  }

  params_list = [p_g1, p_g2, p_g3, p_g4, p_g5, p_g6, p_g7, p_g8, p_g9]

  # Model training and evaluation loop
  for reg, grid in zip(models_list, params_list):
      model_name.append(type(reg).__name__)
      gs = GridSearchCV(reg, param_grid=grid, n_jobs=-1, cv=5,
                        scoring='neg_mean_squared_error')

      gs.fit(X_train, y_train)

      best_reg = reg.__class__(**gs.best_params_)

      print(f'{type(reg).__name__}: Best Params: {gs.best_params_}')

      # Evaluate using cross-validation on the training set
      cv_results = gs.cv_results_
      best_score = np.min(cv_results['mean_test_score'])

      print(f'Cross-validated MSE: {-best_score:.2f}')

      best_reg.fit(X_train, y_train)

      y_pred_val = best_reg.predict(X_val)
      y_pred_train = best_reg.predict(X_train)
      y_pred_test = best_reg.predict(X_test)

      # Validation metrics
      print('Validation MSE:', round(mean_squared_error(y_pred_val, y_val), 2))
      print('Validation R^2:', round(r2_score(y_val, y_pred_val), 2))
      print('Validation MAE:', round(mean_absolute_error(y_val, y_pred_val), 2))

      RMSE_validation.append(round(mean_squared_error(y_pred_val, y_val), 2))
      R2_validation.append(round(r2_score(y_val, y_pred_val), 2))
      MAE_validation.append(round(mean_absolute_error(y_val, y_pred_val), 2))

      # Training metrics
      print('Train MSE:', round(mean_squared_error(y_pred_train, y_train), 2))
      print('Train R^2:', round(r2_score(y_train, y_pred_train), 2))
      print('Train MAE:', round(mean_absolute_error(y_train, y_pred_train), 2))

      RMSE_train.append(round(mean_squared_error(y_pred_train, y_train), 2))
      R2_train.append(round(r2_score(y_train, y_pred_train), 2))
      MAE_train.append(round(mean_absolute_error(y_train, y_pred_train), 2))

      # Test metrics
      print('Test MSE:', round(mean_squared_error(y_pred_test, y_test), 2))
      print('Test R^2:', round(r2_score(y_test, y_pred_test), 2))

      RMSE_test.append(round(mean_squared_error(y_pred_test, y_test), 2))
      R2_test.append(round(r2_score(y_test, y_pred_test), 2))
      MAE_test.append(round(mean_absolute_error(y_test, y_pred_test), 2))

      print(' ')